In [1]:
import os
import sys
sys.path.insert(0, "functions/")
import extract 

In [2]:
from radiomics import setVerbosity
setVerbosity(60)

In [3]:
import pandas as pd
import numpy as np
import time

import SimpleITK as sitk
from tqdm import tqdm

# Loading Data 

In [4]:
os.chdir('..')
extractionPath = os.path.join(os.getcwd(),'data','extractionData')
imagePath = os.path.join(extractionPath, 'images_raw')
maskPath  = os.path.join(extractionPath, 'masks')
labelPath = os.path.join(extractionPath, 'label.csv')

## Mask

In [5]:
maskPath_lst = [os.path.join(maskPath, file) for file in os.listdir(maskPath)]
maskPath_lst.sort()

In [6]:
#Getting pixel array for every mask
maskNumpy_lst = [sitk.GetArrayFromImage(sitk.ReadImage(file)) for file in maskPath_lst]

#All masks must be binary masks with only 1 and 0.
normalize = lambda mask_object, m_max: np.where(m_max == mask_object, 1, 0)
maskNumpy_lst = [normalize(mask_np, mask_np.max()) for mask_np in maskNumpy_lst]

#Transforming to mask object
maskObject_lst = [sitk.GetImageFromArray(mask_np) for mask_np in maskNumpy_lst]

## Image 

In [7]:
imagePath_lst = []
# r=root, d=directories, f = files
for r, d, f in os.walk(imagePath):
    for file in f:
        if '.dcm' in file:
            imagePath_lst.append(os.path.join(r, file))

In [8]:
imageObject_lst = [sitk.ReadImage(file) for file in imagePath_lst]

#Ordering by Patient ID number in order to match with the masks
imageObject_lst = sorted(imageObject_lst, key = lambda i: i.GetMetaData('0010|0020'))

## Labels

In [9]:
label_df = pd.read_csv(labelPath)
#Somente para dar o Merge.
label_df['ID'] = label_df['ID'].astype(int)
label_df['SEVERE'] = label_df['SEVERE'].astype(int)
label_df = label_df[['ID','SEVERE']] 

# Extracting

In [10]:
dataFrame = pd.DataFrame()
feat = ["FO", "GLCM", "GLSZM", "GLRLM", "NGTDM", "GLDM"]
binValues_lst = np.arange(32,272,16)

for binValue in binValues_lst:
    
    print(f'Bin: {binValue}')
    
    time.sleep(1)
    start = time.time()   
    for j in tqdm(np.arange(len(maskObject_lst))):
        
        case_name = str(imageObject_lst[j//8].GetMetaData('0010|0020').strip()) + '_' + str(j%8 + 1) + '_' + str(binValue)
        
        aux = extract.extractFeatures(imageObject_lst[j//8], 
                                      maskObject_lst[j], 
                                      case_name, 
                                      binCount= binValue, features = feat, reduce = True)

        dataFrame = pd.concat([dataFrame, aux], ignore_index=True)
        
    end = time.time()
    print(f'Time:{round(end - start, 3)} seconds')
    print('#'*30)

Bin: 32


100%|████████████████████████████████████████████████████████████████████████████████| 776/776 [01:54<00:00,  6.76it/s]


Time:114.786 seconds
##############################
Bin: 48


100%|████████████████████████████████████████████████████████████████████████████████| 776/776 [02:02<00:00,  6.35it/s]


Time:122.233 seconds
##############################
Bin: 64


...

Time:699.4 seconds
##############################
Bin: 240


100%|████████████████████████████████████████████████████████████████████████████████| 776/776 [13:40<00:00,  1.06s/it]


Time:820.334 seconds
##############################
Bin: 256


100%|████████████████████████████████████████████████████████████████████████████████| 776/776 [16:28<00:00,  1.27s/it]

Time:988.859 seconds
##############################


## Cleaning

In [11]:
dataFrame[['ID', 'MASK','BIN']] = dataFrame['File Name'].apply(lambda x: pd.Series(x.split('_')))
dataFrame['ID'] = dataFrame['ID'].astype(int)
dataFrame['MASK'] = dataFrame['MASK'].astype(int)
dataFrame['BIN'] = dataFrame['BIN'].astype(int)

In [12]:
dataFrame = dataFrame.merge(label_df, on = 'ID', how = 'inner')
col = dataFrame.columns[-4:].tolist()+dataFrame.columns[1:-4].tolist()
dataFrame = dataFrame[col]

## Saving

In [13]:
dataFrame.to_parquet(os.path.join(os.getcwd(), 'data', 'modelData','modelData.parquet'))